# Tutorial

In this tutorial we fairly compare a number of ensemble methods using EI's built in nested cross-validation implementation, and show how predictions can be made with the selected final model. We then show how we can intepret the model by calculating feature rankings.

### Performance analysis and selection of ensemble methods

First of all let's import some `sklearn` models, `EnsembleIntegration` and some additional ensemble methods:

In [1]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
import pandas as pd
from eipy.ei import EnsembleIntegration
from eipy.additional_ensembles import MeanAggregation, CES

Next make some dummy "multi-modal" data from the breast cancer dataset:

In [2]:
import numpy as np
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data = load_breast_cancer()
feature_names = data.feature_names
X = data.data
X = pd.DataFrame(X, columns=feature_names)  # add feature names
y = np.abs(data.target - 1)  # make "malignancy" the positive class rather than "benign"

X_1 = X.iloc[:, 0:10]
X_2 = X.iloc[:, 10:]

X_1_train, X_1_test, y_train, y_test = train_test_split(X_1, y, test_size=0.2, random_state=3, stratify=y)
X_2_train, X_2_test, _, _ = train_test_split(X_2, y, test_size=0.2, random_state=3, stratify=y)

Create dictionaries containing data modalities:

In [3]:
data_train = {
                "Modality_1": X_1_train,
                "Modality_2": X_2_train
                }

data_test = {
                "Modality_1": X_1_test,
                "Modality_2": X_2_test
                }

Define metrics of interest.

In [4]:
from sklearn.metrics import f1_score, roc_auc_score, matthews_corrcoef

metrics = {
            'f_max': f1_score,  # max suffix in the metric key will return the maximized f1_score
            'auc': roc_auc_score,
            'mcc_max': matthews_corrcoef  # max suffix in the metric key will return the maximized matthews_corrcoef
            }

Define base predictors:

In [5]:
base_predictors = {
                    'ADAB': AdaBoostClassifier(),
                    'XGB': XGBClassifier(),
                    'DT': DecisionTreeClassifier(),
                    'RF': RandomForestClassifier(), 
                    'GB': GradientBoostingClassifier(),
                    'KNN': KNeighborsClassifier(),
                    'LR': LogisticRegression(),
                    'NB': GaussianNB(),
                    'MLP': MLPClassifier(),
                    'SVM': SVC(probability=True),
}

Initialise Ensemble Integration:

In [6]:
EI = EnsembleIntegration(
                        base_predictors=base_predictors,
                        k_outer=5,
                        k_inner=5,
                        n_samples=1,
                        sampling_strategy="undersampling",
                        sampling_aggregation="mean",
                        n_jobs=-1,
                        metrics=metrics,
                        random_state=38,
                        project_name="breast_cancer",
                        model_building=True,
                        )

Fit base predictors on each modality. Remember to include the unique modality name.

In [28]:
for name, modality in data_train.items():
    EI.fit_base(modality, y_train, modality_name=name)

Training base predictors on Modality_1...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%




Training base predictors on Modality_2...
        
... for ensemble performance analysis...


Generating meta training data: |██████████|100%
Generating meta test data: |██████████|100%



... for final ensemble...


Generating meta training data: |██████████|100%
Training final base predictors: |██████████|100%


We can check the cross validated performance of each base predictor on each modality with `base_summary`:

In [29]:
EI.base_summary['metrics']

modality       Modality_1                                                    \
base predictor       ADAB        DT        GB       KNN        LR       MLP   
f_max            0.918129  0.890173  0.908555  0.836013  0.877778  0.825397   
auc              0.977802  0.914345  0.977564  0.917214  0.970320  0.924272   
mcc_max          0.868874  0.823152  0.872648  0.759582  0.804551  0.740383   

modality                                               Modality_2            \
base predictor        NB        RF       SVM       XGB       ADAB        DT   
f_max           0.893491  0.912181  0.844156  0.923077   0.952663  0.902857   
auc             0.972621  0.974592  0.935459  0.979608   0.984128  0.926109   
mcc_max         0.820740  0.858614  0.775230  0.875993   0.919982  0.843132   

modality                                                                    \
base predictor        GB       KNN        LR       MLP        NB        RF   
f_max           0.932153  0.896755  0.930636  0.895522  0.927536  0.941896   
auc             0.984004  0.958524  0.988524  0.963715  0.987255  0.987513   
mcc_max         0.901928  0.835469  0.897343  0.834643  0.892473  0.891587   

modality                            
base predictor       SVM       XGB  
f_max           0.872093  0.943284  
auc             0.966130  0.984727  
mcc_max         0.794510  0.894557

Now let's define some meta models for stacked generalization. We add an "S." prefix to the keys of stacking algorithms.

In [11]:
from eipy.metrics import max_min_score

def fmax_scorer(y_true, y_pred):  # create fmax scorer
     return max_min_score(y_true=y_true, 
                          y_pred=y_pred, 
                          metric=f1_score, 
                          pos_label=1,
                          max_min='max'
                          )[0]

meta_predictors = {     
                    'Mean' : MeanAggregation(),
                    'CES' : CES(scoring=fmax_scorer),
                    'S.ADAB': AdaBoostClassifier(),
                    'S.XGB': XGBClassifier(),
                    'S.DT': DecisionTreeClassifier(),
                    "S.RF": RandomForestClassifier(), 
                    'S.GB': GradientBoostingClassifier(),
                    'S.KNN': KNeighborsClassifier(),
                    'S.LR': LogisticRegression(),
                    'S.NB': GaussianNB(),
                    'S.MLP': MLPClassifier(),
                    'S.SVM': SVC(probability=True),
}

Fit meta models:

In [12]:
EI.fit_meta(meta_predictors=meta_predictors)

Analyzing ensembles: |██████████|100%
Training final meta models: |██████████|100%


Check the meta summary with `meta_summary`:

In [16]:
EI.meta_summary['metrics']

,Mean,CES,S.ADAB,S.XGB,S.DT,S.RF,S.GB,S.KNN,S.LR,S.NB,S.MLP,S.SVM
f_max,0.936047,0.949853,0.928793,0.945122,0.935294,0.953846,0.940828,0.943620,0.953488,0.940171,0.952941,0.946746
auc,0.988710,0.990691,0.981424,0.984727,0.948349,0.985614,0.979567,0.975697,0.988338,0.975728,0.983240,0.979794
mcc_max,0.902221,0.906691,0.892746,0.915756,0.896698,0.930686,0.910506,0.910557,0.925388,0.903839,0.929660,0.916910


The MLP stacking algorithm has the best $\text{F}_\text{max}$ performance (the preferred metric for imbalanced datasets) so let's select it as our final model.

### Predictions on unseen data

Since we ran EI with `model_building=True`, we can make predictions. Let's predict the test set and apply the $\text{F}_\text{max}$ threshold calculated during training:

In [18]:
y_pred = EI.predict(X_dict=data_test, meta_model_key='S.MLP')

threshold = EI.meta_summary['thresholds']['S.MLP']['f_max']

y_pred[y_pred>=threshold] = 1
y_pred[y_pred<threshold] = 0

print(y_pred)

[0. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0.
 1. 1. 1. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 0. 0. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0.
 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0.]



### Interpreting the final model

We now use `PermutationInterpreter` to interpret the final MLP stacked generalization model. Let's first import `PermutationInterpreter` and our chosen metric, and initialise the interpreter:

In [26]:
from eipy.interpretation import PermutationInterpreter

interpreter = PermutationInterpreter(EI=EI,
                                     metric=fmax_scorer,
                                     meta_predictor_keys=['S.MLP'])

Calculate feature importance scores:

In [27]:
interpreter.rank_product_score(X_dict=data_test, y=y_test)

Interpreting ensembles...



Calculating local feature ranks: |██████████|100%
Calculating local model ranks: |██████████|100%

Calculating combined rank product score...
... complete!


We can now inspect the most important features for model prediction:

In [31]:
ranking_dataframe = interpreter.ensemble_feature_ranking['S.MLP']

ranking_dataframe

,modality,feature,RPS,feature rank,ensemble method
3,Modality_1,mean area,0.077000,1.0,S.MLP
23,Modality_2,worst area,0.122250,2.0,S.MLP
21,Modality_2,worst texture,0.154250,3.0,S.MLP
22,Modality_2,worst perimeter,0.165000,4.0,S.MLP
7,Modality_1,mean concave points,0.179500,5.0,S.MLP
27,Modality_2,worst concave points,0.181000,6.0,S.MLP
13,Modality_2,area error,0.199250,7.0,S.MLP
6,Modality_1,mean concavity,0.204500,8.0,S.MLP
26,Modality_2,worst concavity,0.228375,9.0,S.MLP
1,Modality_1,mean texture,0.229500,10.0,S.MLP
